In [1]:
import pandas as pd

# CSV dosyasını okuma
csv_file = 'Churn_Modelling.csv'
df = pd.read_csv(csv_file)

# Excel dosyasına yazma
excel_file = 'Churn_Modelling.xlsx'
df.to_excel(excel_file, index=False)

print(f'{csv_file} başarıyla {excel_file} olarak kaydedildi.')

Churn_Modelling.csv başarıyla Churn_Modelling.xlsx olarak kaydedildi.


## 1-1 Görevi

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Veriyi Churn_Modelling.xlsx veriyi yükleme yapıcak
df = pd.read_excel("Churn_Modelling.xlsx")

# O1: Sözel kategorik tipteki özellikleri sayısallaştırma.
# Geography sütununu ve Gender Sütununu pd.Categorical sayesinde kategorik veri tipine çevirir.
df['Geography'] = pd.Categorical(df['Geography']).codes
df['Gender'] = pd.Categorical(df['Gender']).codes

# O2: Boş değerleri en çok görülen değer ile doldur.
# Bunu yaparken DataFrame'in içindeki boş olan yerleri doldurur bunu yaparken mod() işleminden yararlanır çünkü mod işlemi en sık görülen değerdir.
# iloc[0] ifadesi'de aldığımız modu DataFrame'deki ilk satırı almamıza olanak sağlar ama mod değeri yani aynı tekrar'a sahip olan değerler varsa ilk en çok tekrarlanan değeri almamızı sağlar.
df = df.fillna(df.mode().iloc[0])

# O3: Alakasız kolonları sil.
# Burada ise df.drop() yani drop fonksiyonu en üstteki satır'da olan [RowNumber,CustomerId,Surname] saütunlarını kaldırır.
# Bu sütunların kaldırılma sebepi ise RowNumber ve CustomerId  yukarıdan aşağıya artan değer gösterir 1'er 1'er artar bu yüzden farklı değerler olduğu için sınıflandırmaya uygun olmaz o yüzden bu sütunları silmemiz gerekir.
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# O4: Min-Max normalizasyonu
# Min-Max normalizasyonu için Min yani minimum ve Max yani Maximum değerler belirlenir sayısal değerleri bir aralığa koymak için kullanılır.
# Pd.DataFrame() ile veriyi normal veriyi pandas'a çevirmemizi sağlar.
# Scaler orjinal veriyi dataframe(df) olmasını sağlar 
# Fit sayesinde veriyi normal haline çevirme işlemi yapar.
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# O5: Veriyi random olarak karıştır ve kNN algoritması ile sınıflandırma
# np.random.seed(42) seed'rastgele sayı üretmemiz için olanak sağlar.
np.random.seed(42)  
# df_normalicez_ fonksiyonu bize DatFrame'in uzunluğunu gösterir.
# df_normalized.iloc[ ] Rastele sıralanmış satırları satırları bulmamızı sağlar.
# np.random.permutation Sayı aralığını rastgele karıştırmamızı sağlar.
df_shuffled = df_normalized.iloc[np.random.permutation(len(df_normalized))]

# kNN algoritması ile sınıflandırma ve başarı oranını hesaplama
# k_values'da listenin içine bize görevde verilen değerlerin atamasını sağlar.
k_values = [1, 3, 5, 7, 9, 11]
# Farklı komşu sayıları için  k'nın sağladığı doğruluk değerlerini listenin içine atar.
accuracies = []

for k in k_values:
    # np.random.seed(42) seed'rastgele sayı üretmemiz için olanak sağlar.
    np.random.seed(42)  # Her bir k için farklı rastgele tohum kullan
    
    # Aşağıdaki kod satırında drop ile Bağımsız x değişkeninde Exited sütununu siler
    X = df_shuffled.drop('Exited', axis=1)
    # Aşağıdaki kod satırında ise y değişkeninde drop olmadığı için y değişkeni exited sütununu içermiş olur.
    y = df_shuffled['Exited']
    # %30'unu test_size yani test eğitim seti olarak ayırır geriye kalan %70 ise Eğitim veri seti olur.
    # random state ise rasgele sayı üretimini 42 ile başlatır .
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    # n_neighbors k'nın komşu sayısını bulmasını sağlar .
    # Aşağıdaki kod satırı metric değerini öklid'e eşitliyoruz yani (euclidean) değerine eşitlemiş oluyoruz öklid uzunluğunu hesaplıyor.
    knn_classifier = KNeighborsClassifier(n_neighbors=k,metric="euclidean")
    # Eğitim seti X_train ve Y_train üzerinde sağlanır X_daha önceden drop olayında söylediğim gibi bağımsız değişkendir Y_train ise Bağımlı değişkendir.
    knn_classifier.fit(X_train, y_train)
    # predict sayesinde bağımsız değişkenlere bağlanarak tahminlerini yapar.
    predictions = knn_classifier.predict(X_test)
    # Modelin tahminleri ile gerçek değerleri karşılaştırarak doğruluk değeri hesaplanır. Doğruluk değeri, tüm örnekler içinde doğru tahmin edilen örneklerin oranını ifade eder.
    accuracy = accuracy_score(y_test, predictions)
    # Değeri eklemeyi sağlar.
    accuracies.append(accuracy)

# Sonuçları k'nın eşitlik değerleri [1,3,5,7,9,11] için sonuçları yazdırır.
for k, acc in zip(k_values, accuracies):
    print(f'k={k}, Başarı: {acc}')

# Toplam başarı oranını yazdırmaması sağlar.
total_accuracy = accuracy_score(y_test, predictions)
print(f'Toplam Başarı: {total_accuracy}')


k=1, Başarı: 0.7853333333333333
k=3, Başarı: 0.8003333333333333
k=5, Başarı: 0.8073333333333333
k=7, Başarı: 0.81
k=9, Başarı: 0.8116666666666666
k=11, Başarı: 0.8106666666666666
Toplam Başarı: 0.8106666666666666


### 2-1 Görevi

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
# Veriyi Churn_Modelling.xlsx veriyi yükleme yapıcak
df = pd.read_excel("Churn_Modelling.xlsx")

# O1: Sözel kategorik tipteki özellikleri sayısallaştırma.
# Geography sütununu ve Gender Sütununu pd.Categorical sayesinde kategorik veri tipine çevirir.
df['Geography'] = pd.Categorical(df['Geography']).codes
df['Gender'] = pd.Categorical(df['Gender']).codes

# O2: Boş değerleri en çok görülen değer ile doldur.
# Bunu yaparken DataFrame'in içindeki boş olan yerleri doldurur bunu yaparken mod() işleminden yararlanır çünkü mod işlemi en sık görülen değerdir.
# iloc[0] ifadesi'de aldığımız modu DataFrame'deki ilk satırı almamıza olanak sağlar ama mod değeri yani aynı tekrar'a sahip olan değerler varsa ilk en çok tekrarlanan değeri almamızı sağlar.
df = df.fillna(df.mode().iloc[0])

# O3: Alakasız kolonları sil.
# Burada ise df.drop() yani drop fonksiyonu en üstteki satır'da olan [RowNumber,CustomerId,Surname] saütunlarını kaldırır.
# Bu sütunların kaldırılma sebepi ise RowNumber ve CustomerId  yukarıdan aşağıya artan değer gösterir 1'er 1'er artar bu yüzden farklı değerler olduğu için sınıflandırmaya uygun olmaz o yüzden bu sütunları silmemiz gerekir.
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# O4: Min-Max normalizasyonu
# Min-Max normalizasyonu için Min yani minimum ve Max yani Maximum değerler belirlenir sayısal değerleri bir aralığa koymak için kullanılır.
# Pd.DataFrame() ile veriyi normal veriyi pandas'a çevirmemizi sağlar.
# Scaler orjinal veriyi dataframe(df) olmasını sağlar 
# Fit sayesinde veriyi normal haline çevirme işlemini yapıcaktır.
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# O5: Veriyi random olarak karıştır ve kNN algoritması ile sınıflandırma
# np.random.seed(42) seed'rastgele sayı üretmemiz için olanak sağlar.
np.random.seed(42)
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# Her bir test örneği için k adet en yakın komşuyu kullanarak sınıflandırma yapıcaktır.
k = 3
accuracies = []

for _ in range(1):  # 100 farklı random durum için deneme yapabilirsiniz.
    # %70'ini train_set yani eğitim seti olarak ayırır, geriye kalan %30 ise test_seti olur.
    X_train, X_test, y_train, y_test = train_test_split(df_shuffled.drop('Exited', axis=1), df_shuffled['Exited'], test_size=0.3, random_state=42)
    
    # kNN algoritması için her bir test örneği için k adet en yakın komşuyu bulma yani (k=3) bulmasını sağlıyor.
    predicted_classes = []
    for index, test_row in X_test.iterrows():
        # Her bir sınıf için k adet en yakın komşuyu bulma
        positive_neighbors = X_train[y_train == 1].apply(lambda row: np.linalg.norm(row - test_row), axis=1).nsmallest(k)
        # Exited'sütununda 1 ve 0 değerleri var değeri 1 olanları pozitif olarak alıyor.
        # Negatif olanları ise 0 olduğu zaman alıyor.

        negative_neighbors = X_train[y_train == 0].apply(lambda row: np.linalg.norm(row - test_row), axis=1).nsmallest(k)
        
        # Pozitif ve negatif sınıfın k adet en yakın komşularının ortalamasını bulmasını sağlıyor.
        positive_mean = positive_neighbors.index.to_numpy()
        negative_mean = negative_neighbors.index.to_numpy()
        
        # Test örneğini pozitif ve negatif sınıfın ortalama vektörleri ile karşılaştırarak tahmin yapma
        positive_distance = np.linalg.norm(test_row - X_train.loc[positive_mean].mean())
        negative_distance = np.linalg.norm(test_row - X_train.loc[negative_mean].mean())
        predicted_class = 1 if positive_distance < negative_distance else 0
        predicted_classes.append(predicted_class)
    
    # Test seti için başarı oranını hesaplama işlemini yapar.
    accuracy = accuracy_score(y_test, predicted_classes)
    accuracies.append(accuracy)

# Sonuçları yazdırma işlemini gerçekleştirir.
print(f'Toplam Başarı: {np.mean(accuracies)}')


Toplam Başarı: 0.6986666666666667


### 3-1 Görevi

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy import stats

# Veriyi Churn_Modelling.xlsx veriyi yükleme yapıcak
df = pd.read_excel('Churn_Modelling.xlsx')

# Kategorik özellikleri sayısallaştırma
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

# Boş değerleri en çok görülen değer ile doldurma işlemini yapar.
np.random.seed(42)
missing_indices = np.random.choice(df.index, size=(int(0.1 * len(df))), replace=False)
df.loc[missing_indices, 'CreditScore'] = np.nan
df.fillna(df.mode().iloc[0], inplace=True)

# Sınıflandırma görevi ile alakasız kolonları silme
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

# Min-Max normalizasyonu
df = (df - df.min()) / (df.max() - df.min())


# Aşağıdaki kod satırında drop ile Bağımsız x değişkeninde Exited sütununu siler
X = df.drop('Exited', axis=1)
# Aşağıdaki kod satırında ise y değişkeninde drop olmadığı için y değişkeni exited sütununu içermiş olur.
y = df['Exited']
# Veriyi eğitim ve test setlerine ayırma oranına göre ayırma işlemi.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# K değerlerini içeren liste.
k_values = [1, 3, 5, 7, 9]

# Tahmin array listeleri.
tahmin_1 = [] 
tahmin_3 = []
tahmin_5 = []
tahmin_7 = []
tahmin_9 = []

for i in range(len(X_test)):
    # k=1
    knn_classifier_1 = KNeighborsClassifier(n_neighbors=1)
    knn_classifier_1.fit(X_train, y_train)
    y_pred_1 = knn_classifier_1.predict(X_test.iloc[i:i+1])
    tahmin_1.append(y_pred_1[0])

    # k=3
    knn_classifier_3 = KNeighborsClassifier(n_neighbors=3)
    knn_classifier_3.fit(X_train, y_train)
    y_pred_3 = knn_classifier_3.predict(X_test.iloc[i:i+1])
    tahmin_3.append(y_pred_3[0])

    # k=5
    knn_classifier_5 = KNeighborsClassifier(n_neighbors=5)
    knn_classifier_5.fit(X_train, y_train)
    y_pred_5 = knn_classifier_5.predict(X_test.iloc[i:i+1])
    tahmin_5.append(y_pred_5[0])

    # k=7
    knn_classifier_7 = KNeighborsClassifier(n_neighbors=7)
    knn_classifier_7.fit(X_train, y_train)
    y_pred_7 = knn_classifier_7.predict(X_test.iloc[i:i+1])
    tahmin_7.append(y_pred_7[0])

    # k=9
    knn_classifier_9 = KNeighborsClassifier(n_neighbors=9)
    knn_classifier_9.fit(X_train, y_train)
    y_pred_9 = knn_classifier_9.predict(X_test.iloc[i:i+1])
    tahmin_9.append(y_pred_9[0])

# Tahmin array'lerini yatay olarak birleştirme işlemini gerçekleştiriyor yatay denildiği için vstack kullanıyoruz.
tahmin_matrix = np.vstack([tahmin_1, tahmin_3, tahmin_5, tahmin_7, tahmin_9]).T

# Her bir satırdaki en çok görülen değeri final tahmin olarak hesaplama
final_predictions_mode = np.array(stats.mode([tahmin_1, tahmin_3, tahmin_5, tahmin_7, tahmin_9], axis=0)[0]).flatten()
accuracy_mode = accuracy_score(y_test, final_predictions_mode)
print(f"Mod Başarı Oranı: {accuracy_mode * 100:.2f}%")



Mod Başarı Oranı: 82.17%
